In [1]:
import time
import pandas as pd
import numpy as np
import requests
import json
import seaborn as sns
import matplotlib.pyplot as plt
import re
from selenium import webdriver
import bs4
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [3]:
client_id = 'AJb44uAUgBS5u1gIdz0P'
client_secret = 'ThMXBzW9HG'

url = 'https://openapi.naver.com/v1/search/news?'

header = {"X-Naver-Client-Id":client_id,"X-Naver-Client-secret":client_secret}

symbol = '카카오' # 종목
word_list = ['','뉴스','발표','제품','기업','개발','기술','최신','정보','브랜드','이미지','서비스'] # 검색어
display = 100 # 1~100 고정
start = 1 # 1~1000 고정
sort = 'sim' # sim(정확도), date(시간순)

title_list = [] # 기사제목
link_list = [] # 링크
date_list = [] # 날짜
desc_list = [] # 간단설명
for word in word_list:
    keyword = symbol + ' ' + word # 종목 + 검색어
    start = 1
    for _ in range(11):
        params = {'query':keyword , 'display':display, 'start':start, 'sort':sort}
        req = requests.get(url, params = params, headers = header)
        js = req.json()
        title_list += [js['items'][i]['title'] 
                       for i in range(100) 
                       if 'news.naver.com/main' in js['items'][i]['link'] # 네이버뉴스만 추출
                       if symbol in js['items'][i]['title']] # 기사제목에 종목이름 포함만 추출
        link_list += [js['items'][i]['link'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        date_list += [js['items'][i]['pubDate'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        desc_list += [js['items'][i]['description'] 
                      for i in range(100) 
                      if 'news.naver.com/main' in js['items'][i]['link'] 
                      if symbol in js['items'][i]['title']]
        start += 100
        if start == 1001: # 1100개 풀크롤링
            start = 1000
    print(f'{keyword} 검색 뉴스기사 크롤링 완료')

카카오  검색 뉴스기사 크롤링 완료
카카오 뉴스 검색 뉴스기사 크롤링 완료
카카오 발표 검색 뉴스기사 크롤링 완료
카카오 제품 검색 뉴스기사 크롤링 완료
카카오 기업 검색 뉴스기사 크롤링 완료
카카오 개발 검색 뉴스기사 크롤링 완료
카카오 기술 검색 뉴스기사 크롤링 완료
카카오 최신 검색 뉴스기사 크롤링 완료
카카오 정보 검색 뉴스기사 크롤링 완료
카카오 브랜드 검색 뉴스기사 크롤링 완료
카카오 이미지 검색 뉴스기사 크롤링 완료
카카오 서비스 검색 뉴스기사 크롤링 완료


In [4]:
# date_list 포멧 변경
mon_dict = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06','Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'}    
ymd_list = []
for date in date_list:
    date_split = date.split()
    y = date_split[3]
    m = mon_dict[date_split[2]]
    d = date_split[1]
    ymd = f'{y}-{m}-{d}'
    ymd_list.append(ymd)

week_dict = {'Mon,':'월요일','Tue,':'화요일','Wed,':'수요일','Thu,':'목요일','Fri,':'금요일','Sat,':'토요일','Sun,':'일요일'}
week_list = []
for date in date_list:
    week = week_dict[date.split()[0]]
    week_list.append(week)

In [5]:
df = pd.DataFrame() # DF 생성
df['날짜'] = ymd_list
df['요일'] = week_list
df['기사제목'] = title_list
df['간단설명'] = desc_list
df['링크'] = link_list

df = df.drop_duplicates(['링크']) # 링크 중복 제거
df = df.drop_duplicates(['기사제목']) # 제목 중복 제거

df['날짜'] = pd.to_datetime(df['날짜']) # ymd열 타입 datetime으로 변경
df = df.sort_values(by=['날짜'], axis=0, ascending = False) # 최신순 정렬
df = df.reset_index(drop=True) # 인덱스 리셋

In [20]:
df.tail(600)

,날짜,요일,기사제목,간단설명,링크
1548,2022-03-02,수요일,<b>카카오</b>T <b>브랜드</b>택시 마스터 어워즈…110명 시상,<b>카카오</b>모빌리티는 가맹운송사업자 KM솔루션·DGT모빌리티와 '2022 <...,https://news.naver.com/main/read.naver?mode=LS...
1549,2022-03-02,수요일,2022 <b>카카오</b> T <b>브랜드</b>택시 마스터 어워즈 개최,<b>카카오</b>모빌리티는 ‘2022 <b>카카오</b> T <b>브랜드</b>택...,https://news.naver.com/main/read.naver?mode=LS...
1550,2022-03-02,수요일,"<b>카카오</b>모빌리티, 우수 <b>카카오</b>T <b>브랜드</b>택시 기사...","<b>카카오</b>모빌리티는 지난달 28일 가맹운송사업자인 KM솔루션, DGT모빌리...",https://news.naver.com/main/read.naver?mode=LS...
1551,2022-03-02,수요일,"<b>카카오</b> VX ‘<b>카카오</b>골프예약’, 최초 예약 고객 및 신규 ...","먼저, '<b>카카오</b>골프예약' 이용 내역이 없는 회원에게는 '선결제 골프장 ...",https://news.naver.com/main/read.naver?mode=LS...
1552,2022-03-02,수요일,"<b>카카오</b>모빌리티, '<b>카카오</b>T <b>브랜드</b>택시 마스터 ...","<b>카카오</b>모빌리티가 가맹운송사업자인 KM솔루션, DGT모빌리티와 함께 ‘2...",https://news.naver.com/main/read.naver?mode=LS...
...,...,...,...,...,...
2143,2020-06-01,월요일,"<b>카카오</b>키즈 <b>최신</b> 앱, 삼성전자 모바일기기 키즈홈 메인에 기...",<b>카카오</b>키즈는 삼성전자의 <b>최신</b> 스마트폰과 태블릿PC ‘키즈홈...,https://news.naver.com/main/read.naver?mode=LS...
2144,2020-05-18,월요일,인천시 &quot;<b>카카오</b>톡으로 <b>최신</b> 치매 정보 받아보세요&...,가장 대중적인 SNS매체인 <b>카카오</b>톡 채널을 활용해 <b>최신</b> 치...,https://news.naver.com/main/read.naver?mode=LS...
2145,2019-10-29,화요일,"<b>카카오</b>키즈, 삼성 '키즈홈'과 제휴… <b>최신</b> 단말기 기본 탑재",이번 제휴에 따라 삼성 <b>최신</b> 단말기에 ‘<b>카카오</b>키즈앱’이 기...,https://news.naver.com/main/read.naver?mode=LS...
2146,2019-10-28,월요일,<b>카카오</b>키즈-삼성전자 '키즈홈' 제휴...<b>최신</b> 단말기에 기본 탑재,<b>카카오</b>키즈는 삼성전자와 어린이 사용자를 위한 '키즈홈' 서비스 제휴를 ...,https://news.naver.com/main/read.naver?mode=LS...


In [7]:
desc_list

['글로벌 웹툰 플랫폼 <b>카카오</b>웹툰이 27일 인도네시아에서 서비스를 시작한다고 <b>카카오</b>엔터테인먼트가 밝혔다. <b>카카오</b>엔터는 2018년 12월 인도네시아 웹툰업체 네오바자르를 인수한 뒤 현지 웹툰 서비스인 웹코믹스를... ',
 '<b>카카오</b>엔터테인먼트는 글로벌 웹툰 플랫폼 <b>카카오</b>웹툰을 인도네시아에 출시한다고 27일 밝혔다. <b>카카오</b>엔터는 2018년 12월 인도네시아 웹툰업체 네오바자르를 인수한 뒤 현지 웹툰 서비스인 웹코믹스를 <b>카카오</b>페이지... ',
 '<b>카카오</b>모빌리티가 반려동물 이동 서비스 ‘<b>카카오</b>T 펫’을 서울과 인천, 경기 전 지역에서 선보인다고 27일 밝혔다. 베타 서비스로 운행을 시작한 뒤 기사 공급망을 넓혀, 운행 가능 지역을 확대한다는 계획이다. 정식... ',
 '<b>카카오</b>엔터테인먼트(대표 이진수, 김성수)가 글로벌 웹툰 플랫폼 <b>카카오</b>웹툰을 인도네시아에 론칭, 글로벌 공략에 불을 붙인다. 한국의 프리미엄 지식재산권(IP)과 <b>카카오</b>엔터테인먼트의 글로벌 성공 노하우를... ',
 '네이버, <b>카카오</b>, T맵 등을 통해서 주변 외래진료센터를 검색할 수 있다”고 밝혔습니다. 재택치료 중인 확진자는 네이버 지도, <b>카카오</b>맵, T맵 등의 검색 창에 ‘외래진료센터’ 검색을 해서 대면 진료가 가능한 근처 병... ',
 '기사내용 요약주얼리·핸드백·시계·지갑 등 200여개 제품 구매 가능 <b>카카오</b>톡으로 명품 주얼리 브랜드 불가리를 선물할 수 있게 됐다. <b>카카오</b>는 <b>카카오</b>톡 선물하기에 불가리가 공식 입점했다고 27일 밝혔다. 불가리가... ',
 '오늘부터 재택치료 중 대면진료가 가능한 외래진료센터 정보를 네이버나 <b>카카오</b> 등 포털에서 검색할 수 있게 됩니다. 중앙사고수습본부는 오늘 정례 브리핑에서, &quot;앞으로 재택치료 중인 확진자는 발열 등의 증상이... ',

In [14]:
print(link_list[3])

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=106&oid=021&aid=0002511402
